# Leon blijf van deze code af, jou definitie van clean code voldoet niet aan mijn standaard, grt Fronk

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, MaxPooling3D
from keras.layers import BatchNormalization
import numpy as np
import os
import cv2
import glob

In [ ]:
train_dir = 'cnn_data_train'
train_path = os.path.join(train_dir, '*g')

train_imgsNom = glob.glob(train_path)
train_labels = [int(name[-5]) for name in train_imgsNom]
train_imgs = np.array([np.array(cv2.imread(img, cv2.IMREAD_GRAYSCALE)) for img in train_imgsNom])

In [ ]:
test_dir = 'cnn_data_test'
test_path = os.path.join(test_dir, '*g')

test_imgsNom = glob.glob(test_path)
test_labels = np.array([int(name[-5]) for name in test_imgsNom])
test_imgs = np.array([np.array(cv2.imread(img, cv2.IMREAD_GRAYSCALE)) for img in test_imgsNom])

For image loading and preprocessing see:
https://keras.io/preprocessing/image/

In [ ]:
num_classes = 2

# input image dimensions
img_rows, img_cols = 60, 60

print('train_imgs shape:', train_imgs.shape)
print(train_imgs.shape[0], 'train samples')
print(test_imgs.shape[0], 'test samples')

# convert class vectors to binary class matrices
train_label = keras.utils.to_categorical(train_labels, num_classes)
# val_label = keras.utils.to_categorical(val_labels, num_classes)
test_label = keras.utils.to_categorical(test_labels, num_classes)

In [ ]:
### Basic CNN ###

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu', 
                 input_shape=(img_rows,img_cols,3)))
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer = 'random_normal'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu', kernel_initializer = 'random_normal'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax', kernel_initializer = 'random_normal'))

In [ ]:
### The model used by DNT ###

model2 = Sequential()
model2.add(Conv2D(4, kernel_size=(5, 5),
                 activation='relu',
                 input_shape=(img_rows,img_cols,3)))
model2.add(Conv2D(5, (3, 3), activation='relu', kernel_initializer = 'random_normal'))
model2.add(Dropout(0.3))
model2.add(Flatten())
model2.add(Dense(16, activation='relu', kernel_initializer = 'random_normal', input_shape=(24*24*5,)))
model2.add(Dense(num_classes, activation='softmax', kernel_initializer = 'random_normal'))

In [ ]:
### Our improvement of DNT ###

model3 = Sequential()
model3.add(Conv2D(4, kernel_size=(5, 5),
                 activation='relu',
                 input_shape=(img_rows,img_cols,3)))
model3.add(Conv2D(5, (3, 3), activation='relu', kernel_initializer = 'random_normal'))
model3.add(MaxPooling2D(pool_size=(4,4), padding='valid'))
model3.add(Dropout(0.3))
model3.add(Flatten())
model3.add(Dense(16, activation='relu', kernel_initializer = 'random_normal', input_shape=(24*24*5,)))
model3.add(Dense(num_classes, activation='softmax', kernel_initializer = 'random_normal'))

# Na 30 epochs: 0.84

In [ ]:
### B-Human ball-detector model ###

model4 = Sequential()
model4.add(BatchNormalization())
model4.add(Conv2D(4, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(img_rows,img_cols,3),
                 kernel_initializer = 'random_normal'))
model4.add(MaxPooling2D(pool_size=(2,2), padding='valid'))
model4.add(BatchNormalization())
model4.add(Conv2D(8, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(img_rows,img_cols,3),
                 kernel_initializer = 'random_normal'))
model4.add(MaxPooling2D(pool_size=(2,2), padding='valid'))
model4.add(BatchNormalization())
model4.add(Conv2D(8, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(img_rows,img_cols,3),
                 strides=(2, 2),
                 kernel_initializer = 'random_normal'))
model4.add(MaxPooling2D(pool_size=(2,2), padding='valid'))
model4.add(Flatten())
model4.add(BatchNormalization())
model4.add(Dense(num_classes, activation='softmax', kernel_initializer = 'random_normal'))

# Na 30 epochs 0.8842592570516798

In [ ]:
batch_size = 128
epochs = 30

current_model = model4

current_model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

current_model.fit(train_imgs, train_label,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_split=.2)
score = current_model.evaluate(test_imgs, test_label, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
# Save current model
current_model.save('current_cnn.h5')

# Leons garbage code

In [ ]:
from texttable import Texttable

pred_test =  current_model.predict(test_imgs)
predT = np.array(pred_test[:,0]) < 0.5
realT = test_labels == 1
print(sum(predT == realT))
print(len(predT))
print("accuracy is:",accuracy(predT,realT))
# print(predT)
# print(realT)
def accuracy(predT, realT):
    return sum(predT == realT)/ len(realT)    

def precision(predT,realT):
    # if it says robot is it robot?
    return sum(predT and realT)/predT

# def recall()

def evalMatrix(predT,realT):
    t = Texttable()
    t.add_rows([['Pred\Real', 'True', 'False '], ['True', sum(predT & realT), sum(predT & (~realT)) ], ['False',  sum(~predT & realT), sum(~ predT & ~ realT) ]])
    print(t.draw())
evalMatrix(np.array(predT),np.array(realT))
# print(~realT)

In [ ]:
leon = np.array([True,True,False,False])
Hm = np.array([True,False,False,True])
print(leon & Hm)
print(165/216)

In [ ]:
# model2.compile(loss=keras.losses.categorical_crossentropy,
#               optimizer=keras.optimizers.Adadelta(),
#               metrics=['accuracy'])

# model2.fit(train_imgs, train_label,
#           batch_size=batch_size,
#           epochs=epochs,
#           verbose=1,
#           validation_split=.2)
# score = model2.evaluate(test_imgs, test_label, verbose=0)
# print('Test loss:', score[0])
# print('Test accuracy:', score[1])

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
np.array(test_imgsNom)[predT & (~realT)]
# print(test_imgsNom)

In [ ]:
print(test_labels)
plt.imshow(test_imgs[0])